<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Neural_Architecture_Search_(NAS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Define the SimpleNAS class
class SimpleNAS(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleNAS, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)  # First fully connected layer
        self.fc2 = nn.Linear(128, 256)  # Second fully connected layer
        self.fc3 = nn.Linear(256, output_dim)  # Third fully connected layer
        self.activation = nn.ReLU()  # ReLU activation function
        self.layers = [self.fc1, self.fc2, self.fc3]  # List of all layers
        self.selected_layers = self.layers  # Example: All layers are selected

    def forward(self, x):
        for layer in self.selected_layers:
            x = self.activation(layer(x))  # Apply activation function after each layer
        return x

# Example usage
model = SimpleNAS(input_dim=784, output_dim=10)  # Instantiate the model
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)  # Define the optimizer